In [1]:
import pandas as pd
import numpy as np
import re
# import matplotlib as mpl # optional (here)
# import matplotlib.pyplot as plt
# import seaborn as sns # Optional, will only affect the color of bars and the grid
# import pyodbc
# from datetime import datetime

#Graph
# from itertools import combinations
# import random
import networkx as nx
from pyvis.network import Network
# import plotly.io as pio
# pio.renderers.default = 'notebook'
from IPython.display import display
from IPython.core.display import HTML
# import plotly.graph_objs as go
# from ipywidgets import widgets, interactive
# import imgkit
import ast

/Users/vikassoni/.pyenv/versions/3.7.4/lib/python3.7/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
df = pd.read_csv('trial_data_dva.csv')

In [3]:
df.head()

,Unnamed: 0,SourceTag,Tags,Year,Month,Date,Tags_parsed
0,0,python,<python><ctypes>,2018,1,2018-01-01,"['python', 'ctypes']"
1,1,python,<python><django><nginx>,2018,1,2018-01-01,"['python', 'django', 'nginx']"
2,2,python,<python><csv><web-scraping><scrapy>,2018,1,2018-01-01,"['python', 'csv', 'web-scraping', 'scrapy']"
3,3,python,<python><events><python-decorators><discord.py>,2018,1,2018-01-01,"['python', 'events', 'python-decorators', 'dis..."
4,4,python,<python><events><python-decorators><discord.py>,2018,1,2018-01-01,"['python', 'events', 'python-decorators', 'dis..."


In [4]:
df_small = df[:10].copy()
df_small = df_small[['SourceTag', 'Tags', 'Tags_parsed']]
df_small

,SourceTag,Tags,Tags_parsed
0,python,<python><ctypes>,"['python', 'ctypes']"
1,python,<python><django><nginx>,"['python', 'django', 'nginx']"
2,python,<python><csv><web-scraping><scrapy>,"['python', 'csv', 'web-scraping', 'scrapy']"
3,python,<python><events><python-decorators><discord.py>,"['python', 'events', 'python-decorators', 'dis..."
4,python,<python><events><python-decorators><discord.py>,"['python', 'events', 'python-decorators', 'dis..."
5,python,<python><object><import><scripting><maya>,"['python', 'object', 'import', 'scripting', 'm..."
6,python,<python><mysql><compiler-errors>,"['python', 'mysql', 'compiler-errors']"
7,python,<python><pyqt><pyqt4><pyqt5>,"['python', 'pyqt', 'pyqt4', 'pyqt5']"
8,python,<python-3.x><dictionary>,"['python-3.x', 'dictionary']"
9,python,<python><exception><pydoc>,"['python', 'exception', 'pydoc']"


In [5]:
def extract_tags(text):
    tags = re.findall(r'<(.*?)>', text)
    return tags

In [6]:
# def Network_Maker(df):
#     df_2 = df.copy()
#     df_2['new_tags'] = df_2['Tags'].apply(extract_tags)
#     df_2.drop(['Tags'],inplace=True,axis=1)
#     df_2.rename({'new_tags':'Tags'},inplace=True,axis=1)
    
    
#     ## Get unique values of tags from df
#     unique_tags = df_2["Tags"].explode().unique()
    
#     # Create an empty list to store the smaller DataFrames
#     dfs = []

#     # Iterate over each unique tag
#     for tag in unique_tags:
#         # Create a smaller DataFrame for the current tag
#         temp_df = df_2[df_2['Tags'].apply(lambda x: tag in x)]
#         temp_df['SourceTag2'] = tag
        

#         # Append the smaller DataFrame to the list
#         dfs.append(temp_df)
        
#         # Remove the tag value from the list in df_2['Tags']
#         df_2['Tags'] = df_2['Tags'].apply(lambda x: [i for i in x if i != tag])
    
#     # Concatenate the DataFrames vertically into a single DataFrame
#     dfs = pd.concat(dfs)
    
    
#     # Drop rows where df['Tags'] is equal to df['SourceTag2']
#     dfs = dfs[~(dfs['Tags'].apply(len) == 1)]
    
#     # Reset the index of the DataFrame
#     dfs = dfs.reset_index(drop=True)
#     dfs = dfs[['SourceTag2', 'Tags']]
#     dfs = dfs.rename(columns={"SourceTag2": "SourceTag"})

    
#     display(dfs)


#     exploded_df = dfs.explode('Tags')
#     filtered_df = exploded_df[exploded_df['Tags'] != exploded_df['SourceTag']]
    
#     return filtered_df

In [7]:
# exploded_df = Network_Maker(df_small)
# display(exploded_df.head(10))
# display(exploded_df.tail(10))

In [8]:
def Network_Maker(df):
    df_2 = df.copy()
    df_2['new_tags'] = df_2['Tags'].apply(extract_tags)
    df_2.drop(['Tags'],inplace=True,axis=1)
    df_2.rename({'new_tags':'Tags'},inplace=True,axis=1)
    
    ####Creating dataframes for unique tags and concatenating#####

    ## Get unique values of tags from df
    unique_tags = df_2["Tags"].explode().unique()
    
    # Create an empty list to store the smaller DataFrames
    dfs = []

    # Iterate over each unique tag
    for tag in unique_tags:
        # Create a smaller DataFrame for the current tag
        temp_df = df_2[df_2['Tags'].apply(lambda x: tag in x)]
        temp_df['SourceTag2'] = tag
        

        # Append the smaller DataFrame to the list
        dfs.append(temp_df)
        
        # Remove the tag value from the list in df_2['Tags']
        df_2['Tags'] = df_2['Tags'].apply(lambda x: [i for i in x if i != tag])
    
    # Concatenate the DataFrames vertically into a single DataFrame
    dfs = pd.concat(dfs)
    
    
    # Drop rows where df['Tags'] is equal to df['SourceTag2']
    dfs = dfs[~(dfs['Tags'].apply(len) == 1)]
    
    # Reset the index of the DataFrame
    dfs = dfs.reset_index(drop=True)
    dfs = dfs[['SourceTag2', 'Tags']]
    dfs = dfs.rename(columns={"SourceTag2": "SourceTag"})

    
#     display(dfs)


    exploded_df = dfs.explode('Tags')
    filtered_df = exploded_df[exploded_df['Tags'] != exploded_df['SourceTag']]
    
    ##############data frame creation ends########
    
    #######Network Graph Code begins here##########
    
    data = pd.DataFrame({'source': filtered_df['SourceTag'],
                         'target': filtered_df['Tags']})
    data['target'] = data['target'].astype("string")

    edges = data.groupby(by=['source',"target"]).size().reset_index(name='weight')
    edges = edges.nlargest(n=30,columns=['weight'])

    g = nx.from_pandas_edgelist(edges, 'source','target','weight')
    net = Network(
        notebook=True,
        cdn_resources="in_line",
        bgcolor="#222222",
        font_color="white",
        select_menu=True
        # height="2000px",
        # width="2000px",
    )
    for node in g.nodes:
        node_size = g.degree[node]*2
        net.add_node(node, size=node_size)

    for source, target, weight in g.edges.data('weight'):
        net.add_edge(source,target,value=weight)
    # nodes = list(set([*edges.source, *edges.target]))
        edges_list = edges.values.tolist()


###     Enable physics for draggable nodes
    net.toggle_physics(True)
    return net.show("stackoverflow_network.html")


In [9]:
network_graph_small = Network_Maker(df_small)
network_graph_small

stackoverflow_network.html


/Users/vikassoni/.pyenv/versions/3.7.4/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [10]:
network_graph_trial = Network_Maker(df)
network_graph_trial

/Users/vikassoni/.pyenv/versions/3.7.4/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


stackoverflow_network.html
